# Taxi Networks simulation

In this notebook, we will demonstrate code that simulates the behavior of taxis
in queueing networks. We are modeling this using a Jackson network, which models
taxis as customers traveling in the network. Taxis queue at each station $i$,
and are served by customers arriving at a rate of $\mu_i$. After a customer
gets on a taxi, it travels to another station $j$ with probability
$r_{ij}$. We assume the taxis 

In [48]:
import numpy as np
import scipy.linalg as la
import scipy.stats as sps
import networkx as nx
from matplotlib import pyplot as plt
from simulation import *
%matplotlib inline
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [41]:
test_full_nw = full_network(3, 1.1, 1, 10)
test_full_nw.write_graphviz('full.dot')
test_nw = l_to_r_attack(3, 1.1, 1, 10, 1)
test_nw.write_graphviz('l2r.dot')

In [50]:
for node in test_nw.graph.values():
    if isinstance(node, StationNode):
        print node.r
    else:
        print node.rid


{0.11: 0.7380952380952381, 0.12: 0.2619047619047619}
{1.12: 0.7380952380952381, 1.1: 0.2619047619047619}
{2.11: 0.5, 2.1: 0.5}
0
1
2
0
1
2


In [66]:
test_nw = l_to_r_attack(3, 1.1, 1, 10, 1)
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]
test_nw.tick()
print [(nid, node.n) for nid, node in test_nw.graph.items()]


[(0, 10), (1, 10), (2, 10), (1.1, 0), (0.11, 0), (0.12, 0), (2.1, 0), (2.11, 0), (1.12, 0)]
[(0, 10), (1, 10), (2, 9), (1.1, 0), (0.11, 0), (0.12, 0), (2.1, 0), (2.11, 1), (1.12, 0)]
[(0, 10), (1, 10), (2, 8), (1.1, 0), (0.11, 0), (0.12, 0), (2.1, 1), (2.11, 1), (1.12, 0)]
[(0, 10), (1, 9), (2, 8), (1.1, 1), (0.11, 0), (0.12, 0), (2.1, 1), (2.11, 1), (1.12, 0)]
[(0, 10), (1, 9), (2, 7), (1.1, 1), (0.11, 0), (0.12, 0), (2.1, 2), (2.11, 1), (1.12, 0)]


In [77]:
from collections import Counter

test_nw = l_to_r_attack(5, 1.1, 0.1, 100, 0.1)
counts = []
steps = 10000

for _ in range(steps):
    test_nw.tick()
    counts.append(test_nw.get_station_counts()[1])

x = range(steps)
for y in zip(*counts):
    plt.plot(x, y)

print map(np.mean, zip(*counts))

[26.725100000000001, 93.365899999999996, 50.801600000000001, 104.5976, 164.88390000000001]


# Sandbox for math

In [5]:
def mva(v, mu, n):
    dim = len(v)
    gamma = v / mu
    L = np.zeros(dim)
    for k in range(1, n+1):
        L1 = np.ones(dim) + L
        f = k / float(gamma.dot(L1))
        L = f * gamma * L1
    return L

def get_first_evac(m):
    evals, evecs = la.eig(m, right=True)
    max_index = max(enumerate(evals), key=lambda x: abs(x[1]))[0]
    return evecs[:,max_index]

p_real = 1
p_virt = 0.0001
n = 5
m = linear_network(n, p_virt, p_real, 0).to_matrix()
mu = [(p_real + p_virt)]*(n-1) + [p_real]
print np.round(m, 5)

first_evec = get_first_evac(m)
first_evec /= first_evec[-1]

print first_evec / mu
first_evec = first_evec.astype(float)
print mva(first_evec, mu, n*30)

[[ 0.       0.24998  0.24998  0.24998  0.25   ]
 [ 0.25007  0.       0.24998  0.24998  0.25   ]
 [ 0.24998  0.25007  0.       0.24998  0.25   ]
 [ 0.24998  0.24998  0.25007  0.       0.25   ]
 [ 0.24998  0.24998  0.24998  0.25007  0.     ]]
[ 0.99984002+0.j  0.99992000+0.j  0.99992001+0.j  0.99992001+0.j
  1.00000000+0.j]
[ 29.93806828  29.99995281  29.99995777  29.99995777  30.06206337]


/usr/lib/python2.7/site-packages/IPython/kernel/__main__.py:27: ComplexWarning: Casting complex values to real discards the imaginary part


In [6]:
m = np.array([[0,0,0,0,0.25], [1,0,0,0,0.25], [0,1,0,0,0.25], [0,0,1,0,0.25], [0,0,0,1,0]])

first_evec = get_first_evac(m)
first_evec /= first_evec[-1]
print first_evec


[ 0.25+0.j  0.50+0.j  0.75+0.j  1.00+0.j  1.00+0.j]
